In [4]:
import os
import pandas as pd

In [20]:
def parse_line(line:str):
  current = 0
  lineResult = ""
  while current < len (line):
    objeto = line.find('"{', current)
    lista = line.find('"[', current)
    if objeto > -1 and (lista > -1 and objeto < lista or lista == -1):
      lineResult += line[current:objeto+1]
      objetoEnd = line.find('}"', objeto+1)
      if objetoEnd == -1:
        raise Exception("Error al parsear objeto en el delivery: ", line.split(",")[0], " cursor: ", current)
      lineResult += line[objeto+1:objetoEnd+1].replace('"', "'") + '"'
      current = objetoEnd + 2
    elif lista > -1:
      lineResult += line[current:lista+1]
      listaEnd = line.find(']"', lista+1)
      if listaEnd == -1:
        raise Exception("Error al parsear objeto en el delivery: ", line.split(",")[0], " cursor: ", current)
      lineResult += line[lista+1:listaEnd+1].replace('"', "'") + '"'
      current = listaEnd + 2
    else:
      lineResult += line[current:]
      break
  return lineResult

In [22]:
files = os.listdir('../data')
for file in filter(lambda x: x.endswith('.csv'), files):
  input_path = os.path.join(os.path.abspath('../data'), file)
  output_path = os.path.join(os.path.abspath('../data/parsed'), file)
  with open(input_path, encoding='utf-8') as f:
    with open(output_path, 'w', encoding='utf-8') as g:
      g.write(f.readline())
      for line in f:
        try:
          g.write(parse_line(line))
        except Exception as e:
          print(e)

('Error al parsear objeto en el delivery: ', 't-amabrt-1787948', ' cursor: ', 2364)
('Error al parsear objeto en el delivery: ', 't-amabrt-1791223', ' cursor: ', 2127)
('Error al parsear objeto en el delivery: ', 't-amacnf-1152637', ' cursor: ', 2370)
('Error al parsear objeto en el delivery: ', 't-amaste-1248572', ' cursor: ', 2284)
('Error al parsear objeto en el delivery: ', 't-casrde-290021', ' cursor: ', 1570)
('Error al parsear objeto en el delivery: ', 't-petz-3203755', ' cursor: ', 2500)


In [30]:
files = os.listdir('../data/parsed')
deliveries = pd.DataFrame()
events = pd.DataFrame()
for file in  filter(lambda x: x.endswith('.csv'), files):
  input_path = os.path.join(os.path.abspath('../data/parsed'), file)
  df = pd.read_csv(input_path, encoding='utf-8')
  if file.startswith('deliveries'):
    deliveries = pd.concat([deliveries, df])
  else:
    events = pd.concat([events, df])


c:\Python39\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (4,69) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Python39\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
# Avoiding double database reads
deliveries.drop_duplicates(subset=['id'], inplace=True)
events.drop_duplicates(subset=['id'], inplace=True)

In [33]:
deliveries.to_csv(os.path.join(os.path.abspath('../data/parsed'), 'deliveries.csv'), index=False)
events.to_csv(os.path.join(os.path.abspath('../data/parsed'), 'events.csv'), index=False)